# KEGG enrichment of stable genes

This notebooks looks at the group of most and least stable genes and performs a KEGG enrichment analysis to determine if there are any KEGG pathways that are significantly over-represented in our most or least stable gene sets.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest
from scripts import paths, utils, annotations

In [2]:
# Load KEGG pathway data
pao1_pathway_filename = "https://raw.githubusercontent.com/greenelab/adage/7a4eda39d360b224268921dc1f2c14b32788ab16/Node_interpretation/pseudomonas_KEGG_terms.txt"

In [3]:
pao1_pathways = annotations.load_format_KEGG(pao1_pathway_filename)
print(pao1_pathways.shape)
pao1_pathways.head()

(169, 2)


,1,2
0,,
KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,10,"{PA3925, PA2553, PA3589, PA2000, PA4785, PA173..."
KEGG-Pathway-pae00071: Fatty acid degradation,32,"{PA5020, PA5427, PA3299, PA5349, PA0447, PA345..."
KEGG-Pathway-pae00903: Limonene and pinene degradation,9,"{PA1748, PA3014, PA1737, PA3331, PA1821, PA489..."
KEGG-Pathway-pae00380: Tryptophan metabolism,27,"{PA0704, PA2147, PA0202, PA0447, PA1585, PA461..."
KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,16,"{PA3925, PA2553, PA3652, PA3589, PA4044, PA365..."


In [4]:
# Load transcriptional similarity df
# These are the subset of genes that we will consider
pao1_similarity_scores_filename = "../5_core_core_analysis/pao1_similarity_scores.tsv"

pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [5]:
pao1_similarity_scores.head()

,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label
PAO1 id,,,,,
PA1642,PA14_43280,0.393037,4.089216e-197,selD,NaN
PA0481,PA14_06280,0.242300,2.526173e-72,NaN,NaN
PA3714,PA14_16350,0.335471,7.698982e-141,NaN,NaN
PA1164,PA14_49350,0.389940,8.702679e-194,NaN,NaN
PA0749,PA14_54600,0.336135,2.004630e-141,NaN,NaN


In [6]:
# Get most and least stable core genes based on label
pao1_most_stable_genes = list(
    pao1_similarity_scores[pao1_similarity_scores["label"] == "most stable"].index
)
pao1_least_stable_genes = list(
    pao1_similarity_scores[pao1_similarity_scores["label"] == "least stable"].index
)

In [7]:
# For each KEGG pathway, perform stat test, save p-values to get corrected p-values, report stats per pathway
def KEGG_enrichment_of_stable_genes(similarity_score_df, gene_list, kegg_df):
    """
    This function performs a KEGG enrichment using most or least stable genes,
    provided in `gene_list`
    """

    all_genes = set(similarity_score_df.index)
    module_genes = set(gene_list)
    not_module_genes = all_genes.difference(module_genes)

    rows = []
    # Find the KEGG pathway with significant over-representation
    for kegg_name in kegg_df.index:
        num_kegg_genes = kegg_df.loc[kegg_name, 1]
        kegg_genes = set(kegg_df.loc[kegg_name, 2])
        not_kegg_genes = all_genes.difference(kegg_genes)

        # Make contingency table
        # ---------------------| most stable  | not most stable
        # in KEGG pathway      | # genes      | # genes
        # not in KEGG pathway  | # genes     | # genes
        module_kegg_genes = module_genes.intersection(kegg_genes)
        not_module_kegg_genes = not_module_genes.intersection(kegg_genes)
        module_not_kegg_genes = module_genes.intersection(not_kegg_genes)
        not_module_not_kegg_genes = not_module_genes.intersection(not_kegg_genes)

        observed_contingency_table = np.array(
            [
                [len(module_kegg_genes), len(not_module_kegg_genes)],
                [len(module_not_kegg_genes), len(not_module_not_kegg_genes)],
            ]
        )
        # Fisher's exact test
        oddsr, pval = scipy.stats.fisher_exact(
            observed_contingency_table, alternative="greater"
        )
        # chi2 test will not accept 0 counts for the contingency table
        # chi2, pval, dof, expected_counts = scipy.stats.chi2_contingency(
        #    observed_contingency_table
        # )
        # print(oddsr, pval)

        rows.append(
            {
                "enriched KEGG pathway": kegg_name,
                "p-value": pval,
                "num shared genes": len(module_kegg_genes),
                "size gene set": len(module_genes),
                "size KEGG pathway": num_kegg_genes,
            }
        )

    enrichment_df = pd.DataFrame(rows)

    # Get corrected pvalues
    (
        reject_,
        pvals_corrected_,
        alphacSidak,
        alphacBonf,
    ) = statsmodels.stats.multitest.multipletests(
        enrichment_df["p-value"].values,
        alpha=0.05,
        method="fdr_bh",
        is_sorted=False,
    )

    enrichment_df["corrected p-value"] = pvals_corrected_

    return enrichment_df

In [8]:
pao1_most_stable_enrichment = KEGG_enrichment_of_stable_genes(
    pao1_similarity_scores, pao1_most_stable_genes, pao1_pathways
)

In [9]:
pao1_least_stable_enrichment = KEGG_enrichment_of_stable_genes(
    pao1_similarity_scores, pao1_least_stable_genes, pao1_pathways
)

In [10]:
print(pao1_most_stable_enrichment.shape)
pao1_most_stable_enrichment.sort_values(by="corrected p-value").head()

(169, 6)


,enriched KEGG pathway,p-value,num shared genes,size gene set,size KEGG pathway,corrected p-value
168,"KEGG-Module-M00178: Ribosome, bacteria",2.649079e-29,39,480,56,2.238471e-27
50,KEGG-Pathway-pae03010: Ribosome,2.649079e-29,39,480,68,2.238471e-27
32,KEGG-Pathway-pae03070: Bacterial secretion system,3.686543e-09,27,480,92,2.076752e-07
114,KEGG-Module-M00332: Type III secretion system,4.839988e-08,11,480,18,2.044895e-06
112,KEGG-Module-M00334: Type VI secretion system,1.717460e-07,16,480,45,5.805015e-06


In [11]:
print(pao1_least_stable_enrichment.shape)
pao1_least_stable_enrichment.sort_values(by="corrected p-value").head()

(169, 6)


,enriched KEGG pathway,p-value,num shared genes,size gene set,size KEGG pathway,corrected p-value
0,KEGG-Pathway-pae00072: Synthesis and degradati...,1.000000,0,202,10,1.0
107,KEGG-Module-M00299: Spermidine/putrescine tran...,1.000000,0,202,12,1.0
108,KEGG-Module-M00222: Phosphate transport system,1.000000,0,202,5,1.0
109,KEGG-Module-M00053: Pyrimidine deoxyribonuleot...,1.000000,0,202,9,1.0
110,KEGG-Module-M00050: Guanine ribonucleotide bio...,0.206327,1,202,6,1.0


In [12]:
# Save
pao1_most_stable_enrichment.to_csv("pao1_most_stable_enrichment.tsv", sep="\t")
pao1_least_stable_enrichment.to_csv("pao1_least_stable_enrichment.tsv", sep="\t")

**Takeaway:**
* There doesn’t appear to be any enriched KEGG pathways in the least stable genes.
    * What does this mean about the role of these least stable core genes? Maybe they are spread across multiple pathways?
* Significantly enriched KEGG pathways include Ribosome (commonly enriched in humans), secretion system, metabolism/Krebs cycle
    * These KEGG pathways represent some of the essential functions for Pa, so it makes sense that they are enriched amongst the set of stable core genes whose transcriptional relationships don’t vary across strains.
    * Only some metabolisms and not others, is that interesting?
